In [1]:
import pandas as pd
from config import *
from db_config import *
session,engine = connect_db()

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
from shutil import copyfile
import os

import matplotlib
font = {'size'   : 21}
matplotlib.rc('font', **font)


                         ...Connecting to the database...                        


In [ ]:
for build in session.query(Build).filter(Build.status == 'building').order_by(Build.build_name):
    print(build.build_name,build.status)

In [ ]:
print(datetime.now(),'Began run')

query_outcomes = "SELECT parts.part_id,parts.status,wells.seq_outcome,wells.plate_type,builds.build_name,wells.misplaced FROM parts \
        INNER JOIN wells ON parts.id = wells.part_id\
        INNER JOIN plates ON wells.plate_id = plates.id\
        INNER JOIN builds ON plates.build_id = builds.id"

query_frag = "SELECT parts.part_id,fragments.fragment_name,twist_orders.sub_name FROM parts\
        INNER JOIN part_frag ON parts.id = part_frag.part_id\
        INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
        INNER JOIN frag_order ON fragments.id = frag_order.frag_id\
        INNER JOIN twist_orders ON twist_orders.id = frag_order.twist_order_id"

query_parts = "SELECT * FROM parts"

df_frag = pd.read_sql_query(query_frag, con=engine)
frags = df_frag.groupby('part_id')['fragment_name'].agg(len)
frags.name = 'Count'
frags = pd.DataFrame(frags).reset_index()
frags_dict = dict(zip(frags.part_id.tolist(),frags.Count.tolist()))
subs_dict = dict(zip(df_frag.part_id.tolist(),df_frag.sub_name.tolist()))

author_dict = []
for file in sorted(glob.glob('../data/*/*.json')):
    with open(file,"r") as json_file:
        data = json.load(json_file)
    author_dict.append([data['gene_id'],data['author']['name']])
author_dict = dict(author_dict)

print(datetime.now(),'Finished frags')

def multiple(x):
    if len(x) == 1:
        x.append('N/A')
    return x

def find_outcome(x):
    if x in df_out_dict.keys():
        return df_out_dict[x]
    else:
        return ['N/A','N/A']
    
def find_build(x):
    if x in df_build_dict.keys():
        return df_build_dict[x]
    else:
        return ['N/A','N/A']
    
def simplify_outcome(x):
    if "mutation" in x:
        return 'cloning_mutation'
    elif "bad" in x:
        return 'sequence_failure'
#     elif x == 'cloning_error':
#         return 'cloning_failure'
    else:
        return x
    
def find_author(x):
    return author_dict[x]

df_res = pd.read_sql_query(query_outcomes, con=engine)
df_res = df_res[df_res.plate_type == 'seq_plate']

df_out = df_res.groupby('part_id')['seq_outcome'].apply(list)
df_out.name = 'Outcomes'
df_out = pd.DataFrame(df_out).reset_index()
df_out.Outcomes = df_out.Outcomes.apply(multiple)
df_out_dict = dict(zip(df_out.part_id.tolist(),df_out.Outcomes.tolist()))

df_build = df_res.groupby('part_id')['build_name'].apply(list)
df_build.name = 'Builds'
df_build = pd.DataFrame(df_build).reset_index()
df_build.Builds = df_build.Builds.apply(multiple)
df_build_dict = dict(zip(df_build.part_id.tolist(),df_build.Builds.tolist()))
print(datetime.now(),'Finished outcomes')

df_parts = pd.read_sql_query(query_parts, con=engine)
print('finished part query')

df_parts = df_parts[df_parts.part_id != 'BBF10K_000745']

df_parts['Fragments'] = df_parts.part_id.apply(lambda x: frags_dict[x])
df_parts['Submission'] = df_parts.part_id.apply(lambda x: subs_dict[x])
df_parts['Order_number'] = df_parts.Submission.apply(lambda name: int(name[-3:]))
df_parts['Outcomes'] = df_parts.part_id.apply(find_outcome)
df_parts['Builds'] = df_parts.part_id.apply(find_build)
print('finished outcome and builds')
df_parts['Attempt_1_Outcome'] = df_parts.Outcomes.apply(lambda x: x[0])
df_parts['Attempt_1_Outcome_G'] = df_parts.Attempt_1_Outcome.apply(simplify_outcome)
df_parts['Attempt_1_Build'] = df_parts.Builds.apply(lambda x: x[0])
df_parts['Attempt_2_Outcome'] = df_parts.Outcomes.apply(lambda x: x[1])
df_parts['Attempt_2_Outcome_G'] = df_parts.Attempt_2_Outcome.apply(simplify_outcome)
df_parts['Attempt_2_Build'] = df_parts.Builds.apply(lambda x: x[1])
df_parts['Length'] = df_parts.seq.apply(len)
df_parts['Author'] = df_parts.part_id.apply(find_author)
print(datetime.now(),'Finished building dataframe')
df_parts

In [ ]:
df_parts.status.value_counts()

In [ ]:
for build in session.query(Build).filter(Build.build_name == 'build029'):
    for plate in build.plates:
        if plate.plate_type != 'assembly_plate':
            continue
        for well in plate.wells:
            print(well.parts.part_id,well.address,well.parts.status)
#             well.parts.status = 'building'
# session.commit()
    

In [ ]:
test = [[1,2],[3,4],[5,6]]
# while len(test) > 0:
num = test.pop()
print(num,test)

In [ ]:
# df_parts.status.value_counts()
df_bt = df_parts[df_parts.Fragments > 1]
# df_bt = df_bt[df_bt.Submission == 'submission010']
df_bt
# df_bt.Submission.value_counts()

In [ ]:
df = df_parts.copy()

# df_parts.status.value_counts()
# df_ordered = df_parts[df_parts.status == 'synthesis_abandoned']
# df_ordered.Order_number.value_counts()

# df_parts = df_parts[df_parts.status == 'received']
df = df[df.part_id == 'BBF10K_001343']
df

In [ ]:
for build in session.query(Build).order_by(Build.build_name).filter(Build.build_name == 'build029'):
    print(build.build_name)
    print(len(build.plates[0].wells))
    for plate in build.plates:
        print(plate.plate_name,plate.plate_type)
        for well in plate.wells:
            print(well.parts.part_id,well.address)

            

In [ ]:
builds = range(16,31)
builds = ['build'+str(build).zfill(3) for build in builds]
print(builds)


query_builds = "SELECT parts.part_id,parts.status,wells.plate_type,builds.build_name,plates.id FROM parts \
        INNER JOIN wells ON parts.id = wells.part_id\
        INNER JOIN plates ON wells.plate_id = plates.id\
        INNER JOIN builds ON plates.build_id = builds.id"

df_builds = pd.read_sql_query(query_builds, con=engine)
df_builds.build_name.value_counts()
df_assembly = df_builds[df_builds.build_name.isin(builds)]

# df_assembly.status.value_counts()
# print(len(df_assembly.part_id.tolist()))
# print(len(df_assembly.part_id.unique().tolist()))
df_ids = df_assembly[['build_name','id']].drop_duplicates()
df_ids
# df_assembly = df_builds[df_builds.plate_type == 'assembly_plate']
# df_building = df_assembly[df_assembly.status == 'building']
# df_building.build_name.value_counts()

dups = pd.concat(g for _, g in df_assembly.groupby("part_id") if len(g) > 1)
dups.build_name.value_counts()


In [ ]:
target_build = 'build022'
query = "SELECT parts.part_id,builds.build_name,part_wells.address as destination,fragments.fragment_name,frag_plates.plate_name,frag_plates.plate_id,frag_wells.address as source,frag_wells.volume FROM parts \
        INNER JOIN wells AS part_wells ON parts.id = part_wells.part_id\
        INNER JOIN plates AS part_plates ON part_wells.plate_id = part_plates.id\
        INNER JOIN builds ON part_plates.build_id = builds.id\
        INNER JOIN part_frag ON parts.id = part_frag.part_id\
        INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
        INNER JOIN wells AS frag_wells ON fragments.id = frag_wells.fragment_id\
        INNER JOIN plates AS frag_plates ON frag_wells.plate_id = frag_plates.id\
        WHERE builds.build_name = '{}'".format(target_build)

df = pd.read_sql_query(query,con=engine)

unique_frag = df[['part_id','fragment_name','destination']].drop_duplicates()
unique_frag

frag_df = unique_frag.groupby('destination').agg(len).part_id
frag_df = frag_df.reset_index()
frag_df = frag_df.rename(columns={'part_id':'frag_num'})
frag_dict = dict(zip(frag_df.destination.tolist(),frag_df.frag_num.tolist()))

df['frag_num'] = df.destination.apply(lambda x: frag_dict[x])
df

unique_df = df[['part_id','destination','frag_num']].drop_duplicates()
unique_df

need_extra = unique_df[unique_df.frag_num > 1]
need_extra

num_reactions = len(df.part_id.unique().tolist())
num_reactions
# total = unique_df.frag_num.sum()
# print(total)


# unique_plates = ['syn_plate021','syn_plate025']
# query_resuspend = "SELECT plates.plate_id,plates.resuspended FROM plates\
#                     WHERE plates.resuspended = 'not_resuspended'\
#                         AND plates.plate_id IN ({})".format(ot.list_to_string(unique_plates))
# resuspended = pd.read_sql_query(query_resuspend,con=engine)
# resuspended
# if len(resuspended) == 0:
#     print('all resuspended')

In [ ]:
for build in session.query(Build).filter(Build.build_name == 'build033'):
    print(build.build_name,build.status)
    for well in build.plates[0].wells:
        print(well.parts.part_id,well.address,well.parts.status)
#         well.parts.status = 'received'
# session.commit()

In [ ]:
df_parts.status.value_counts()

In [ ]:
df_rec = df_parts[df_parts.status == 'received']
df_rec.Fragments.value_counts()

In [ ]:
for part in session.query(Part).filter(Part.status == 'abandoned'):
    print(part.part_id,part.status)
#     part.status = 'synthesis_abandoned'
# session.commit()

In [ ]:
df_copy = df_parts.copy()
df_copy

In [ ]:
df_parts = df_parts[df_parts.Attempt_1_Build.isin(['build012','build013','build014'])]
attempted = ['received','synthesis_abandoned']
cloned = ['sequence_confirmed','cloning_mutation']
total_bp_ordered = df_parts.Length.sum()
total_received = df_parts[df_parts.status != 'synthesis_abandoned'].Length.sum()
total_attempted = df_parts[~df_parts.status.isin(attempted)].Length.sum()
total_cloned = df_parts[df_parts.status.isin(cloned)].Length.sum()
total_perfect = df_parts[df_parts.status == 'sequence_confirmed'].Length.sum()
total_mutant = df_parts[df_parts.status == 'cloning_mutation'].Length.sum()


# df_parts
print(total_bp_ordered)
print(total_received)
print(total_attempted)
print(total_cloned)
print(total_cloned/total_attempted)
print(total_perfect)
print(total_perfect/total_attempted)
print(total_mutant)
print(total_mutant/total_attempted)



In [ ]:
attempted = ['received','synthesis_abandoned']
cloned = ['sequence_confirmed','cloning_mutation']
total_bp_ordered = df_parts.Length.sum()
total_received = df_parts[df_parts.status != 'synthesis_abandoned'].Length.sum()
total_attempted = df_parts[~df_parts.status.isin(attempted)].Length.sum()
total_cloned = df_parts[df_parts.status.isin(cloned)].Length.sum()
total_perfect = df_parts[df_parts.status == 'sequence_confirmed'].Length.sum()
total_mutant = df_parts[df_parts.status == 'cloning_mutation'].Length.sum()
total_gene_cloned = len(df_parts[df_parts.status.isin(cloned)])
largest = df_parts[df_parts.status.isin(cloned)].Length.max()

stats = pd.DataFrame({
    'Ordered':total_bp_ordered,
    'Received':total_received,
    'Attempted':total_attempted,
    'Cloned':total_cloned,
    'Perfect':total_perfect,
    'Mutations':total_mutant
},index=['counts'])

# df_parts
print(total_bp_ordered)
print(total_received)
print(total_attempted)
print(total_cloned)
print(total_cloned/total_attempted)
print(total_perfect)
print(total_perfect/total_attempted)
print(total_mutant)
print(total_mutant/total_attempted)
print(total_gene_cloned)
print(largest)
# stats['Percent_cloned'] = stats.

In [ ]:
ans = '1'
print([int(num) for num in ans.split(' ')])

In [ ]:
num = 3.424444444
new_num = round(num,2)
new_num

In [ ]:
df_parts.groupby('Order_number').Length.agg(sum)

In [ ]:
df_parts.groupby('status').Length.agg(sum)

In [ ]:
for build in session.query(Build).order_by(Build.build_name).filter(Build.build_name == 'build016'):
    print(build.build_name)
    for plate in build.plates:
        for well in plate.wells:
            print(well.parts.part_id,well.address,well.parts.status)
#             well.parts.status = 'building'
# session.commit()

In [ ]:
build = 'build021'
# data = pd.read_csv('{}/builds/{}/{}_trans_map.csv'.format(BASE_PATH,build,build))
# data
# parts = [part for part in session.query(Part).filter(Part.part_id.in_(data.Gene.tolist()))]
# target_build = Build(parts,build_name=build)
# for i,(gene,well) in data.iterrows():
#     print("working on: ", gene,well)
#     gene_obj = session.query(Part).filter(Part.part_id == gene).one()
#     target_build.add_item(gene_obj,well)
for plate in session.query(Build).filter(Build.build_name == build).one().plates:
    for well in plate.wells:
        print(well.parts.part_id,well.address)
# input('continue')
# session.commit()

In [ ]:
build = 'build023'
if len([build for build in session.query(Build).filter(Build.build_name == build)]) == 0:
    print('not found')
else:
    print('found')

In [ ]:
import ot_functions as ot

author_dict = []
for file in sorted(glob.glob('../data/*/*.json')):
    with open(file,"r") as json_file:
        data = json.load(json_file)
    author_dict.append([data['gene_id'],data['author']['name']])
authors = [a for g,a in author_dict]
author_dict = dict(author_dict)

authors = pd.Series(authors).unique()
print('Current authors:\n',authors)
author = ot.request_info('Enter author name: ',select_from=authors)

query = "SELECT parts.part_id,parts.status,builds.build_name,part_wells.address,part_wells.seq_outcome,fragments.fragment_name,frag_plates.plate_name,frag_wells.address FROM parts \
        INNER JOIN wells AS part_wells ON parts.id = part_wells.part_id\
        INNER JOIN plates AS part_plates ON part_wells.plate_id = part_plates.id\
        INNER JOIN builds ON part_plates.build_id = builds.id\
        INNER JOIN part_frag ON parts.id = part_frag.part_id\
        INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
        INNER JOIN wells AS frag_wells ON fragments.id = frag_wells.fragment_id\
        INNER JOIN plates AS frag_plates ON frag_wells.plate_id = frag_plates.id\
        WHERE part_wells.plate_type = 'seq_plate'"

data = pd.read_sql_query(query,con=engine)
data['author'] = data.part_id.apply(lambda x: author_dict[x])
data = data.sort_values('part_id')
data = data[data.author == author]
data

# author_path = '{}/'






In [ ]:
from datetime import datetime

now = str(datetime.now()).split(" ")[0]

data = data.sort_values('build_name')
data
author_path = '{}/authors/{}'.format(BASE_PATH,author)
ot.make_directory(author_path)
data.to_csv('{}/{}_well_locations_{}'.format(author_path,author,now),index=False)



In [ ]:
from datetime import datetime

print(str(datetime.now()).split(" ")[0])

In [ ]:
empty = []
if empty:
    print('true')

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect,update,select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import os
import math
import glob
import json
import numpy as np
import pandas as pd
from datetime import datetime
import getch
import shutil
import sys
import argparse

from config import *
import ot_functions as ot
from db_config import *
session,engine = connect_db()
conn = engine.connect()
    
from moclopy import fixer,fragger
from synbiolib import codon


# query_seqs = "SELECT parts.part_id,parts.part_name,parts.part_type,parts.original_seq,parts.seq,parts.organism FROM parts\
#                 WHERE parts.seq IS NULL"

query_seqs = "SELECT parts.part_id,parts.part_name,parts.part_type,parts.original_seq,parts.seq,parts.organism,fragments.fragment_name FROM parts\
                LEFT JOIN part_frag ON parts.id = part_frag.part_id\
                LEFT JOIN fragments ON part_frag.fragment_id = fragments.id\
                WHERE parts.seq IS NOT Null"

df = pd.read_sql_query(query_seqs,con=engine)
df

In [ ]:
print(df.loc[0].original_seq)
print(df.loc[0].seq)

In [ ]:
# def fragment_sequence(seq,part_type,cloning_enzyme_prefix="GAAGACTT",cloning_enzyme_suffix="GCGTCTTC",synthesis_max=synthesis_max):
df['part_type'] = df.part_type.apply(lambda x: 'prokaryotic_promoter' if x == 'prokaryotic promoter' else x)
df['fragments'] = df.apply(lambda row: fragger.fragment_sequence(row['seq'],row['part_type']), axis=1)

df
    
    

In [ ]:
def optimize_sequences(row):
    if row['part_type'] != 'cds':
        return "not_optimized"
    if row['organism'] == None:
        table = codon.load_codon_table(species='ecoli')
    else:
        table = codon.load_codon_table(species=row['organism'])
    protein_seq = fixer.translate(row['original_seq'])
    optimized = codon.optimize_protein(table,protein_seq)
    
    fixed = fixer.fix_sequence(row['part_id'],optimized)
    
    return fixed

print(datetime.now())
df = df.loc[0:99,:]
df['seq'] = df.apply(optimize_sequences, axis=1)
# df['modified'] = df.apply(lambda row: True if row['original_seq'] == row['seq'] else False,axis=1)
print(datetime.now())
df

In [ ]:
seqs = [part+'_seqs' for part in df.part_id.tolist()]
seq_dict = dict(zip(df.part_id.tolist(),seqs))
print(seq_dict)

for part in session.query(Part).filter(Part.part_id.in_(df.part_id.tolist())):
#     part.seq = seq_dict[part.part_id]
    part.seq = None


# print(seq_dict['FG_001'])

# stmt = update(session.query(Part).all()).values(seq='user #5')

# conn.execute(stmt)

# stmt = select([Part.part_id]).limit(1)
# conn.execute(Part.update().values(seq=stmt))


# session.query(Part).update({Part.seq:seq_dict[Part.part_id]})
session.commit()
# session.query(Part).filter(Part.part_id.in_(df.part_id.tolist())).update({Part.seq: 'test'})


# df.part_id.tolist()
# session.execute(update(parts, values={stuff_table.c.foo: stuff_table.c.foo + 1}))



In [ ]:
sqlalchemy.__version__

In [ ]:
build_names = ['build'+str(num).zfill(3) for num in range(11)]
build_names
for build in session.query(Build).filter(Build.build_name.in_(build_names)).order_by(Build.build_name):
    print(build.build_name)
    for plate in build.plates:
        if plate.plate_type != 'seq_plate':
            continue
        print(plate.plate_name)
        for well in plate.wells:
            print(well.vector,well.address)
            well.vector = 'popen_v1-1'
session.commit()
# for well in session.query(Well).join(Plate,Well.plates).join(Build,Plate.builds).filter(Build.build_name.in_(build_names)):
#     print(well.address)

In [ ]:
build_names = ['build'+str(num).zfill(3) for num in range(30,31)]
build_names

well_ids = [well.id for well in session.query(Well).join(Plate,Well.plates).join(Build,Plate.builds).filter(Build.build_name.in_(build_names)).filter(Plate.plate_type == 'assembly_plate')]
# session.query(Well).join(Plate,Well.plates).join(Build,Plate.builds).filter(Build.build_name.in_(build_names)).filter(Plate.plate_type == 'assembly_plate').update({Well.vector: 'popen_v3-0'})
len(well_ids)
session.query(Well).filter(Well.id == 23784).update({Well.vector: 'popen_v3-0'})
# well_ids
# build_names

In [15]:
import pandas as pd

builds = ['build012','build013','build014']
part_ids = []
for build in builds:
    data = pd.read_csv('{}/builds/{}/{}_seq_plate.csv'.format(BASE_PATH,build,build))
    part_ids += data.Part.tolist()
print(len(part_ids))

query = "SELECT parts.part_id,fragments.fragment_name,parts.part_type,parts.seq,parts.part_name,parts.cloning_enzyme,fragments.seq FROM parts\
            INNER JOIN part_frag ON parts.id = part_frag.part_id\
            INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
            WHERE parts.part_id IN ({})".format(ot.list_to_string(part_ids))

df = pd.read_sql_query(query, con=engine)

df.to_csv('./parts_frags.csv')
# for part,df in df.groupby('part_id'):
#     print(part)
#     for i,row in df.iterrows():
#         print(row.fragment_name)
        
# for part in part_ids:
    
        
        


288
